## Set-Up der Umgebung

In [ ]:
# Install dependencies
!pip install openai wikipedia

In [ ]:
import os
import json
import openai

# Retrieve credentials to access openai
with open('credentials.json') as c:
    data = json.load(c)
    api_key = data['OPENAI_API_KEY']
    
openai.api_key = api_key

if not openai.api_key:
    raise ValueError("No OpenAI API key found! Make sure OPENAI_API_KEY is set as a Codespaces secret.")

print("✅ OpenAI API key loaded successfully!")

## Definiere Default-Tools

Im folgenden werden zwei einfache Tools definiert, die dem Agent von Anfang zur Verfügung stehen:

- **calculator:** Für das Lösen mathematischer Aufgaben, die als `expression` übergeben werden, z.B. `What is 12 * 12?`.
- **wiki_search:** Erlaubt dem Agenten, Zusammenfassungen aus Wikipedia-Artikeln zu erstellen, die sich auf die `query` beziehen.

Die erstellten Tools werden dem `tools`-Dictionary hinzugefügt, welches um neue Tools erweitert werden kann.

In [ ]:
# --- Tools ---

import wikipedia

def calculator(expression: str) -> str:
    """Evaluates basic math expressions."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

def wiki_search(query: str) -> str:
    """Fetches short summaries from Wikipedia."""
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception as e:
        return f"Error: {e}"

tools = {
    "calculator": calculator,
    "wiki": wiki_search,
}

print("✅ Tools ready: calculator(), wiki_search()")

## Definiere die Anweisungen des Agenten

Die Definition der Anweisungen bzw. **System Message** des Agenten werden im folgenden als eigene Funktion ausgelagert, die in der Hauptfunktion des Agenten aufgerufen wird. 

Das gibt uns mehr Flexiblität und hilft dabei, den Code übersichtlicher zu gestalten.

Die System Message setzt sich aus den folgenden Teilen zusammen:
- **tone**: Die Persönlichkeit des Agenten - wenn es keinen Input gibt, gibt es einen Default.
- **tools**: Anhand der *docstrings* (z.B. `"""Evaluates basic math expressions."""`) in den Tool-Funktionen wird automatisiert eine Liste von verfügbaren Tools zusammengestellt.
- **loop**: Der Agent wird angewiesen, stets den *Thought-Action-Observation-Loop* zu befolgen.
- **rules**: Weitere Anweisungen, z.B. *early stopping*, dass der Agent aus dem Loop aussteigt, wenn eine Antwort gefunden wurde.

In [ ]:
# --- System Message ---

def inject_system_message(agent_tone: str = None) -> str:
    # Assign the tone/personality if given, else fall back to default
    tone = agent_tone if agent_tone else "You are an intelligent agent that can reason step by step."

    # Dynamically gather tool descriptions to allow for new tools to be added later on
    tool_descriptions = "\n".join([f"- {name}(): {fn.__doc__ or 'no description'}" for name, fn in tools.items()])

    return f"""{tone}
    You have access to the following tools:
    {tool_descriptions}

    Follow this reasoning format strictly and repeat it, until you find the final answer:
    Thought: describe your reasoning
    Action: choose from the available tools (don't forget to pass the appropriate argument) or ANSWER()
    Observation: output/result of the previous action

    Rules:
    - Use at most ONE tool per step.
    - If you already know the answer, respond with:
    Action: ANSWER(<final answer here>)
    - Do NOT repeat the same tool call multiple times.
    - Stop once you have given the final answer.
    """ 
    
print("✅ Agent system message ready!")

## Definiere die Hauptfunktion des Agenten

Um den Agenten ausführen zu können, ist im folgenden die Funktion `run_agent` vorbereitet. 

Diese nimmt die Anfrage (= *query*) des Users an den Agenten, die maximale Anzahl der *Thought-Action-Observation*-Schritte und einen optionalen *Tone* entgegen. Letzteres wirst du für eine spätere Übung brauchen :).

Bearbeitet wird die Anfrage durch den Aufruf der OpenAI Schnittstelle, welche die folgenden Argumente annimmt:
- **model**: Das zu verwendende LLM - in unserem Fall ist das `gpt-4o-mini` völlig ausreichend.
- **messages**: Hier wird zum einen der *Kontext*, als auch die *Query* angefügt.

Die Antwort (= *response*) des Modells wird dann verarbeitet und je nach Inhalt entsprechend formattiert. 

Der Kontext, der ursprünglich nur aus der *System Message* besteht, wird im weiteren Verlauf um die Ergebnisse bzw. *Observations* aus den vorangegangen Schritten erweitert.

In [ ]:
# --- Agent Function ---

def run_agent(query: str, max_steps: int = 3, agent_tone: str = None):
    print(f"User: {query}\n")

    context = inject_system_message(agent_tone=agent_tone)

    for step in range(1, max_steps + 1):
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": context},
                {"role": "user", "content": f"User question: {query}\nPrevious reasoning:\n{context}"}
            ]
        )

        text = response.choices[0].message.content.strip()
        print(text)

        lines = text.splitlines()
        action_line = next((l for l in lines if l.startswith("Action:")), None)
        if not action_line:
            print("\nNo action detected. Stopping.")
            break

        action = action_line.replace("Action:", "").strip()

        if "ANSWER" in action:
            final = action.split("ANSWER")[-1].strip(" :()")
            print(f"\nFinal Answer: {final}")
            return

        if "(" in action and action.endswith(")"):
            tool_name = action.split("(")[0]
            tool_input = action[len(tool_name) + 1:-1]

            if tool_name in tools:
                observation = tools[tool_name](tool_input)
            else:
                observation = f"Unknown tool: {tool_name}"
            context += f"\n{text}\nObservation: {observation}\n"
        else:
            print("\nCould not parse tool call. Stopping.")
            break

print("✅ Agent main function set up!")

## Probier's aus!

Führe die **Agenten-Funktion** mit verschiedenen Anfragen aus und teste die Verwendung der *Default-Tools* und des *ReAct-Loop*-Formats.

In [ ]:
run_agent("What is 23 * 7?")

In [ ]:
run_agent("What is the square root of 144?")

In [ ]:
run_agent("Who discovered penicillin?")

In [ ]:
run_agent("When was the Eiffel Tower built?")

In [ ]:
# Your code here ...

# 🧩 Übung: Füge dein eigenes Tools hinzu (10 Minuten)

Jetzt bist du dran!

Füge ein **neues Tool** hinzu, das der Agent verwenden kann.  
Einige Ideen:
- Ein `reverse(text)`-Tool, das Wörter umkehrt.
- Ein `coin_flip()`-Tool, das zufällig zwischen „Kopf“ und „Zahl“ wählt.
- Ein Tool `word_count(text)`, das Wörter zählt.

---

### 🧠 Vorgehensweise

1. Definiere dein Tool und beschreibe dessen Funktion mit einem *docstring* (siehe Beispiel in nächster Code-Zelle).
2. Füge dein neues Tool dem `tools`-Dictionary hinzu.
3. Versuche, durch deine Prompts, den Agenten aufzufordern, das neue Tool zu verwenden!

## Beispiel: Word Count Tool

In [ ]:
def reverse(text: str) -> str:
    """Reverses the order of words in the input text."""
    try:
        return " ".join(reversed(text.split()))
    except Exception as e:
        return f"Error: {e}"

# Don't forget to add your new tool to the dictionary
tools["reverse"] = reverse

print("✅ Added new tool: reverse()")

In [ ]:
run_agent("Reverse the following text: 'Artificial intelligence enables creative problem solving!'")

In [ ]:
# Your code here ...

# 🎭 Übung: Verändere die Persönlichkeit des Agenten (5-10 Minuten)

Jetzt kommt das optionale `tone`-Argument der `run_agent()`-Funktion zum Einsatz! Verändere die Persönlichkeit des Agenten.  
Hier ein paar Inspirationen:

- ein **Lehrer**, der seine Schritte im Detail erklärt,  
- ein **Detektiv** dem kein Rätsel zu schwer ist,  
- ein **mittelalterlicher Ritter**, oder
- ein **Digital Native** aus der Generation Z.

---

### 🧠 Vorgehensweise:

1. Erstelle eine Variable für die neue Persönlichkeit und weise diese beim Aufruf von `run_agent` dem Wert `agent_tone=<dein_neuer_tone>` zu:
    > teacher_tone = "You are a curious teacher who explains your reasoning out loud before answering."
2. Probiere verschiedene *tones* aus und beobachte, wie sich der Agent verhält!

## Beispiel: Gen-Z Agent

In [ ]:
# Define a different personality tone than what is currently the default

gen_z_tone = "You are a typical Generation-Z young adult who explains your reasoning out loud (in internet/meme language) before answering."

In [ ]:
# Run the agent with example questions as querys and the Gen-Z tone setting

print("___ Gen-Z Agent ___")
run_agent(query="What is the square root of 144?", agent_tone=gen_z_tone)

print("------")
run_agent(query="Who discovered penicillin?", agent_tone=gen_z_tone)

In [ ]:
# Your code here ...

# 🔍 Übung: Multi-Tool Agent (Optional)

Kannst du den Agenten dazu bringen, **mehrere Tools** zu kombinieren?

Zum Beispiel:
> "What is the population of France divided by the population of Germany?"

Damit könnte der Agent...
1. das `wiki`-Tool zweimal verwenden (einmal pro Land), um die ungefähre Einwohnerzahl zu bestimmen und
2. mithilfe des `calculator`-Tools das Ergebnis ausrechnen.

**Tipp:** Du kannst mehr Schritte im ReAct-Loop zulassen, indem du das Argument `max_steps` in `run_agent()` erhöhst.

In [ ]:
run_agent("What is the population of France divided by the population of Germany?")

In [ ]:
# Your code here ...

# 🏁 Wrap-Up

🎉 Glückwunsch — du hast gerade einen funktionierenden KI-Agenten erstellt!

Du hast gelernt:
- Wie ein Agent mithilfe des **Thought–Action–Observation**-Loop folgert und handelt.  
- Wie **Tools**, zum Beispiel der Taschenrechner oder die Wikipedia-Suche, integriert werden.  
- Wie du das System mit deinen eigenen kreativen Ideen **erweitern** kannst.   

---

💬 **Nächste Schritte**
- Versuche, deine Tools an *echten APIs* anzubinden (e.g., [Open-Meteo](https://open-meteo.com/), [Yahoo Finance](https://github.com/ranaroussi/yfinance)).  
- Erstelle einen *komplexeren Multi-Tool-Agenten* mit [Gedächtnis](https://huggingface.co/docs/smolagents/en/tutorials/memory?utm_source=chatgpt.com).  
- Oder nutze deine gewonnenen Erkenntnisse, um eigene KI-Assistenten in [LibreChat](https://www.librechat.ai/docs/features/agents) zu erstellen.

---

Vielen Dank für deine Teilnahme – und viel Spaß beim weiteren Experimentieren!